In [ ]:
import numpy as np 
import matplotlib.pyplot as plt 
from itertools import combinations
from random import sample
import seaborn as sns
sns.set_style("darkgrid")
plt.rcParams['figure.figsize'] = (14, 6)

In [ ]:
class Graph:
    def __init__(self,N=10):
        self.Nodes = {}
        self._edges_count = 0
        self.edges_list = []
        for i in range(N):
            self.Nodes[i] = []
            
    def add_edge(self,n1,n2):
        if n1 in self.Nodes[n2]:
            return
        self.Nodes[n1].append(n2)
        self.Nodes[n2].append(n1)
        self._edges_count += 1 
        self.edges_list.append((n1,n2))
        
    def get_degree(self,n):
        return len(self.Nodes[n])
    
    def del_edge(self,n1,n2):
        if not (n1 in self.Nodes[n2]):
            return
        
        if n2 in self.Nodes[n1]:
            self.Nodes[n1].remove(n2)
        if n1 in self.Nodes[n2]:
            self.Nodes[n2].remove(n1)
        if (n1,n2) in self.edges_list:
            self.edges_list.remove((n1,n2))
        if (n2,n1) in self.edges_list:
            self.edges_list.remove((n2,n1))   
        
        self._edges_count -= 1
            
    
    def edges_count(self):
        return self._edges_count
    
    
    def cluster_coe(self,n):
        neighbors = self.Nodes[n]
        K_i = len(neighbors)
        if K_i < 2 :
            return 0
        e_i = 0
        
        for first in neighbors:
            for second in neighbors:
                if first in self.Nodes[second]:
                    e_i += 1
                    
            
        e_i /= 2

        return (2*e_i)/(K_i*(K_i-1))


# Erdos-Renyi

In [ ]:
N = 7624
M = 27806

def get_er(N,M):
    graph_er = Graph(N)
    all_possible_edges = list(combinations(range(0,N),2))
    selected_edges = sample(all_possible_edges,M)

    for n1,n2 in selected_edges:
        graph_er.add_edge(n1,n2)
        
    return graph_er
        
    
graph_er = get_er(N,M)

list_of_degrees = []
for node in range(N):
    list_of_degrees.append(graph_er.get_degree(node))


cluster_coe_er = 0
for node in range(N):
    cluster_coe_er += graph_er.cluster_coe(node)

cluster_coe_er /= N
cluster_coe_er = round(cluster_coe_er,8)

plt.subplot(1,2,1)
sns.histplot(list_of_degrees)
plt.ylabel("Count")
plt.xlabel("Degree")
plt.xticks(list(range(20)))
plt.title("Degree Histogram of Erdos-Renyi | Cluster COE: {0}".format(cluster_coe_er))
plt.subplot(1,2,2)


Counts = []
for i in range(30):
    Counts.append(list_of_degrees.count(i))
    
    
plt.scatter(list(range(30)),Counts,color="red")
plt.ylabel("Count")
plt.xlabel("Degree")
plt.title("Degree Histogram of Erdos-Renyi | log-log | Cluster COE: {0}".format(cluster_coe_er))
plt.yscale("log")
plt.xscale("log")
plt.show()


# Small-World

In [ ]:
def get_sw(N,M,_p=0.4):
    Neighbour_index = 1 
    graph_sw = Graph(N)
    stop_flag = False
    # Create Regular circle graph 
    while graph_sw.edges_count() < M:
        Step = Neighbour_index
        
        if Neighbour_index != 4:
            for node in range(N):
                for neighbor in range(node-Step,node):
                    if neighbor < 0:
                        neighbor += N
                    graph_sw.add_edge(neighbor,node)
                    if graph_sw.edges_count() == M :
                        stop_flag = True
                        break
                if stop_flag :
                    break
                
        else:
            remained = N/((M-graph_sw.edges_count())//2)
            for node in range(0,N,int(remained)):
                neighbors = [node+Step,node-Step]
                for neighbor in neighbors:
                    if neighbor < 0:
                        neighbor += N
                    if neighbor >= N :
                        neighbor = neighbor % N                    
                    graph_sw.add_edge(neighbor,node)
                    if graph_sw.edges_count() == M :
                        stop_flag = True
                        break
                if stop_flag :
                    break
            
            break
        Neighbour_index += 1



    # Convert Regular graph to small world (Pick each edge and decide to change the destination)
    Nodes_list = list(range(N))
    P_change_edge = _p
    ChangedEdges = 0 
    for edge_index in range(M):
        edge = graph_sw.edges_list[edge_index]
        s,t = edge
        change = P_change_edge >= np.random.rand()
        if change :
            while True:
                dest = sample(Nodes_list,1)[0]
                if dest in graph_sw.Nodes[s] :
                    continue
                graph_sw.add_edge(s,dest)
                graph_sw.del_edge(s,t)
                break
            ChangedEdges+= 1
            
    return graph_sw

graph_sw = get_sw(N,M)

list_of_degrees = []

for node in range(N):
    list_of_degrees.append(graph_sw.get_degree(node))

cluster_coe_sm = 0

for node in range(N):
    cluster_coe_sm += graph_sw.cluster_coe(node)
    
cluster_coe_sm /= N
cluster_coe_sm = round(cluster_coe_sm,8)

plt.subplot(1,2,1)
sns.histplot(list_of_degrees)
plt.ylabel("Count")
plt.xlabel("Degree")
plt.xticks(list(range(20)))
plt.title("Degree Histogram of Small-World | Cluster COE: {0}".format(cluster_coe_sm))
plt.subplot(1,2,2)


Counts = []
for i in range(20):
    Counts.append(list_of_degrees.count(i))
    
    
plt.scatter(list(range(20)),Counts,color="red")
plt.ylabel("Count")
plt.xlabel("Degree")
plt.title("Degree Histogram of Small-World | log-log | Cluster COE: {0}".format(cluster_coe_sm))
plt.yscale("log")
plt.xscale("log")
plt.show()


# LastFM Asia Social Network

In [ ]:
graph_lf = Graph(N)

lines = list(open("./lasftm_asia/lastfm_asia_edges.csv","r").read().split("\n"))
del lines[0]
del lines[-1]

for line in lines:
    s,t = map(int,line.split(","))
    graph_lf.add_edge(s,t)
    

list_of_degrees = []

for node in range(N):
    list_of_degrees.append(graph_lf.get_degree(node))


uster_coe_lf = 0

for node in range(N):
    uster_coe_lf += graph_lf.cluster_coe(node)
    
uster_coe_lf /= N
uster_coe_lf = round(uster_coe_lf,6)


plt.subplot(1,2,1)
sns.histplot(list_of_degrees)
plt.ylabel("Count")
plt.xlabel("Degree")
plt.title("Degree Histogram of LastFM | Cluster COE: {0}".format(uster_coe_lf))


Counts = []
for i in range(1000):
    Counts.append(list_of_degrees.count(i))
    
plt.subplot(1,2,2)
plt.scatter(list(range(1000)),Counts,color="red")
plt.ylabel("Count")
plt.xlabel("Degree")
plt.title("Degree Histogram of LastFM | log-log | Cluster COE: {0}".format(uster_coe_lf))
plt.yscale("log")
plt.xscale("log")
plt.show()


# Extra works

In [ ]:
import networkx as nx
plt.rcParams['figure.figsize'] = (7, 7)
graph_er_min = get_er(100,400)
_er_x = nx.Graph()
for s,t in graph_er_min.edges_list:
    _er_x.add_edge(s,t)
    

nx.draw_random(_er_x,node_color="#2ae8b8",node_size=100,alpha=0.8,style="--")
plt.title("Sample Erdos-Renyi graph generated by 100 nodes and 400 edges")
plt.show()
    

In [ ]:
plt.rcParams['figure.figsize'] = (7, 7)
graph_er_min = get_sw(50,150)
_er_x = nx.Graph()
for s,t in graph_er_min.edges_list:
    _er_x.add_edge(s,t)
    

nx.draw_circular(_er_x,node_color="#2ae8b8",node_size=100,alpha=0.8,style="--")
plt.title("Sample Samll-World graph generated by 50 nodes and 150 edges")
plt.show()
    

In [ ]:
for p in list(np.arange(0.1,1,0.1)):
    graph_sw = get_sw(N,M,p)
    cluster_coe_sm = 0
    for node in range(N):
        cluster_coe_sm += graph_sw.cluster_coe(node)
        
    cluster_coe_sm /= N
    cluster_coe_sm = round(cluster_coe_sm,8)
    print("P:{0} - Cluster COE:{1}".format(round(p,2),cluster_coe_sm))